In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import sys
sys.path.insert(0, '..//src/')

import pandas as pd
import numpy as np
from datetime import datetime
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import itertools
import main

sns.set_palette(sns.color_palette("colorblind", 15))
plt.rcParams['font.family'] = 'Times New Roman'
font_size = 12
params = {'axes.labelsize': font_size+2,
          'axes.titlesize':font_size+4,
          'legend.fontsize': font_size,
          'xtick.labelsize': font_size,
          'ytick.labelsize': font_size,
          'font.size': font_size}
plt.rcParams.update(params)
sns.set_style("whitegrid")
export_fig_format = "pdf"
fig_format = "png"
dpi = 300
figsize = (12, 3.5)
figsize_map = (16, 8)

markers = ['P', 'o', '^', 's', '*', 'v']
linewidth = 0.1
edgecolor = 'black'

from models import thermal_model, validation
from common import functions, enums, schema
from data import source


In [6]:
path_OAT = main.PATH_ORG / r'General\resources\prediction of overheating data\Weather_16Jun-6Jul_2017.xlsx'
path_measured_IAT =  main.PATH_ORG / r'General\resources\prediction of overheating data\MeasuredTemperature_16Jun-6Jul_2017.xlsx'
path_IES_data = main.PATH_ORG / r'General\resources\prediction of overheating data\Loughborough Output.xlsx'
sheet_name_IES = "IES"

In [7]:
solar_radiation_data = pd.read_excel(path_OAT, sheet_name="1. Weather (16 Jun to 6 Jul)", parse_dates=True, header=[0,1])
solar_radiation_data.set_index(("Label","Units"), inplace=True)

In [ ]:
input_data = pd.read_excel(path_OAT, sheet_name="2. Dry-bulb (16 Jun to 6 Jul)", parse_dates=True)
input_data.set_index("Timestamp", inplace=True)
input_data = input_data.resample('60min').mean()
input_data.index.name = schema.OutputDataSchema.DATEINDEX
input_data.index = input_data.index.tz_localize("utc")
input_data.columns = [schema.OutputDataSchema.OAT]
input_data[schema.OutputDataSchema.SOLARRADIATION] = solar_radiation_data["Total(SPN1).Avg1"].resample('60min').mean().values
input_data.head()

In [15]:
THERMAL_CAPACITY_LEVEL = {
  "low":100,
  "medium":250,
  "high":450,
}


def calculate_thermal_capacity(floor_area:float, level:str="medium") -> float:
  if level in THERMAL_CAPACITY_LEVEL:
    return THERMAL_CAPACITY_LEVEL[level]*floor_area
  else:
    print(f"thermal capacity level {level} not recognized.")
  return 0.

# Create models
def create_dwelling(dwelling_param:dict[str, float])->thermal_model.ThermalModel:
  R = dwelling_param['R']
  C = dwelling_param['C']
  # initial_indoor_air_temperature = dwelling_param['initial_indoor_air_temperature']
  dwelling = thermal_model.ThermalModel(R=R, C=C)
  dwelling.g_t = dwelling_param['g_t']
  return dwelling


def run_simulation(
    param_dwelling: dict,
    weather_data: pd.DataFrame,
) -> pd.DataFrame:
  dwelling = create_dwelling(param_dwelling)
  dwelling.cooling_design_temperature = 0
  data_source = source.SimulationData(dwelling,
                                      weather_data,
                                      timestep_simulation=3600)

  rcmodel_dataf = data_source.create_era5_based_simulation_data(
      estimate_solar_gains=param_dwelling['solar_gains'])
  rcmodel_dataf[schema.DataSchema.TOTALGAINS] = rcmodel_dataf[
      schema.DataSchema.TOTALGAINS]
  dwelling.model_data = rcmodel_dataf
  rcmodel_dataf = dwelling.run_model()
  # resampled_rcmodel_dataf = data_source.resample_modelling_results(
  #     rcmodel_dataf)
  # resampled_rcmodel_dataf.index.name = schema.DataSchema.TIME_HOURS
  functions.print_heating_and_cooling_demand(rcmodel_dataf)

  return rcmodel_dataf


In [10]:
def get_dwelling_parameters(dwelling_name:str, thermal_capacity_level:str, transmittance_factor:float)->dict:
  """Return a dictionnary with the dwelling parameters"""
  floor_area = 3.8+0.8+5.6+4.2+14.4+14+14.2+5.7+13.6

  common_dwelling_parameters = {
    'solar_gains' : True,
    'max_IAT' : 24,
    'g_t' : transmittance_factor, # Transmittance factors for glazing 0.85 (single glazed) to 0.57 (triple glazed), 0.76 double glazed Table 6b p216.
    'thermal_capacity_level':thermal_capacity_level,
    'list_rooms' : ["Living room", "Kitchen", "Front bedroom", "Rear bedroom","Small bedroom"]
  }
  if dwelling_name == 'west dwelling':

    dwelling_parameters = {
      'name' : 'west dwelling',
      'floor_area' : floor_area, # total floor area excluding total floor area
      'R' : 1/(0.223), #K/kW
      'C':calculate_thermal_capacity(floor_area, thermal_capacity_level),  #kJ/K
      'initial_indoor_air_temperature': 22.467323,
      'sheet_names' : ["W.1 Living room (West house)", "W.2 Kitchen (West house)", "W.3 FB (West house)", "W.4 RB (West house)", "W.5 SB (West house)"],
      'volume_rooms' : {
        "W.1 Living room (West house)":34, 
        "W.2 Kitchen (West house)":14.3, 
        "W.3 FB (West house)": 33.6, 
        "W.4 RB (West house)": 34.6, 
        "W.5 SB (West house)": 10},
    }
  elif dwelling_name == 'east dwelling':

    dwelling_parameters = {
      'name' : 'east dwelling',
      'floor_area' : floor_area, # total floor area excluding total floor area
      'R' : 1/(0.216), #K/kW 1/(0.216)
      'C':calculate_thermal_capacity(floor_area, thermal_capacity_level),  #kJ/K
      'initial_indoor_air_temperature': 22.467323,
      'sheet_names' : ["E.1 Living room (East house)", "E.2 Kitchen (East house)", "E.3 FB (East house)", "E.4 RB (East house)", "E.5 SB (East house)"],
      'volume_rooms' : {
        "E.1 Living room (East house)":34, 
        "E.2 Kitchen (East house)":14.3, 
        "E.3 FB (East house)": 33.6, 
        "E.4 RB (East house)": 34.6, 
        "E.5 SB (East house)": 10},
    }
  else:
    dwelling_parameters = {}
  return common_dwelling_parameters | dwelling_parameters

def get_scenario_name(param_dict:dict)->str:
  """Return the scenario name based on the input parameters to the model."""
  return f"{param_dict['name']} ({param_dict['thermal_capacity_level']}, {param_dict['g_t']})"

In [11]:
combined_results = pd.DataFrame()

In [ ]:
transmittance_factors = [0.76] #[0.85, 0.57, 0.76]
thermal_capacity_levels = ['high'] #['low', 'high', 'medium']
combinations = list(itertools.product(transmittance_factors, thermal_capacity_levels))

for temp_transmittance, temp_thermal_capacity in combinations:
  parameters_dwelling_to_model = get_dwelling_parameters('east dwelling', temp_thermal_capacity, temp_transmittance)
  temp_result_simulation = run_simulation(parameters_dwelling_to_model, input_data)
  scenario_name = get_scenario_name(parameters_dwelling_to_model)
  combined_results[scenario_name] = temp_result_simulation[schema.DataSchema.IAT].values

In [ ]:
ies_data = pd.read_excel(path_IES_data, sheet_name=sheet_name_IES, usecols=[x for x in range(2, 11)])
ies_data = ies_data.iloc[:len(temp_result_simulation), :]
ies_data.index = temp_result_simulation.index
ies_data.head()

In [ ]:
### Visualisations

In [ ]:
temp_result_simulation.to_csv(Path(r"../data/results/temp_results_RC_model.csv"))

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
gains_cols = [schema.DataSchema.SOLARGAINS, schema.DataSchema.OCCUPANCYGAINS, schema.DataSchema.APPLIANCESGAINS]
temp_result_simulation[gains_cols].plot(ax=ax)
ax.margins(0, None)
ax.set_ylabel(schema.VisualisationSchema.GAINS)

In [ ]:
viz_results = validation.validation_RC_model(temp_result_simulation, parameters_dwelling_to_model)

In [ ]:
# fig, ax = plt.subplots(figsize=figsize)
# fig, ax = viz_results.plot_measured_data(fig, ax, plot_all_rooms=True)
# ax.get_legend_handles_labels()

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
fig, ax = viz_results.plot_results(fig, ax, plot_all_rooms=True)
ax.legend()
viz_results.calculate_metrics()

In [ ]:
dadawdq

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
fig, ax = viz_results.plot_measured_data(fig, ax)
ax.legend().remove()

for c in combined_results.columns:
    combined_results[c].plot(ax=ax,
                            color='red',
                            linewidth=1)

upper_arrs = combined_results.max(axis=1)
lower_arrs = combined_results.min(axis=1)
x_arr = combined_results.index
ax.fill_between(x_arr, upper_arrs, lower_arrs, alpha=0.3, color='red')

max_cdh = validation.calculate_cooling_degree_hours(upper_arrs, 24)
min_cdh = validation.calculate_cooling_degree_hours(lower_arrs, 24)
print(f'max_cdh = {max_cdh} and min_cdh = {min_cdh}')

In [ ]:
ies_avg_iat = viz_results.weighted_average_measured_IAT_dwelling(ies_data[parameters_dwelling_to_model['sheet_names']])
ies_avg_iat.columns = [enums.DataSource.IES.value]
fig, ax = plt.subplots(figsize=figsize)
fig, ax = viz_results.plot_results(fig, ax)
ies_avg_iat.plot(
    ax=ax,
    kind='line',
    color='blue',
    marker='x',
    markevery=20,
    linewidth=1)
ax.legend()


In [ ]:
viz_results.calculate_metrics()
validation.calculate_cooling_degree_hours(ies_avg_iat[enums.DataSource.IES.value], 24)

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
ax2 = ax.twinx()
dataf[schema.OutputDataSchema.SOLARRADIATION].plot(ax=ax2, color=sns.color_palette()[1])
ax2.set_ylabel(schema.OutputDataSchema.SOLARRADIATION, color=sns.color_palette()[1])
dataf[schema.OutputDataSchema.OAT].plot(ax=ax, color=sns.color_palette()[0])
ax.set_ylabel(schema.OutputDataSchema.OAT, color=sns.color_palette()[0])

ax2.grid(False)
ax.legend().remove()
ax.margins(0, None)

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
dataf[schema.OutputDataSchema.HEATINGSEASON].plot(ax=ax)
ax.set_ylabel(schema.OutputDataSchema.HEATINGSEASON)

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
ax2 = ax.twinx()
resampled_rcmodel_dataf[schema.DataSchema.HEATINGOUTPUT].plot(ax=ax, color=sns.color_palette()[0])
ax.set_ylabel(schema.DataSchema.HEATINGOUTPUT, color=sns.color_palette()[0])
resampled_rcmodel_dataf[schema.DataSchema.IAT].plot(ax=ax2, color=sns.color_palette()[1])
ax2.set_ylabel(schema.DataSchema.IAT, color=sns.color_palette()[1])
ax2.set_ylim(0, 30)
ax2.grid(False)
ax.margins(0, None)

In [ ]:
resampled_rcmodel_dataf.head()

In [ ]:
fig, ax = plt.subplots(figsize=figsize)

measured_IAT[schema.DataSchema.IAT].plot(ax=ax, color='black', linewidth=0.8)
resampled_rcmodel_dataf[schema.DataSchema.IAT].plot(ax=ax, color='red', marker='*', markevery=20, linewidth=1)

for ii, room in enumerate(individual_rooms_IAT.columns):
  individual_rooms_IAT[room].plot(ax=ax, color=sns.color_palette()[ii], linewidth=0.5)
list_rooms = ["Living room", "Kitchen", "Front bedroom", "Rear bedroom", "Small bedroom"]

ax.set_ylabel(schema.DataSchema.IAT)

ax.set_ylim(0, 35)

# ax.grid(False)
ax.margins(0, None)
handles, labels = ax.get_legend_handles_labels()
labels = ["Measured data", "RC model data"] + list_rooms
ax.legend(handles=handles, labels=labels)

MAE_results = functions.calculate_MAE(measured_IAT[schema.DataSchema.IAT], resampled_rcmodel_dataf[schema.DataSchema.IAT])
print(f"The MAE of the model compared to measured data is {MAE_results:.2f} degreeC.")

modelled_cooling_degree_hours = calculate_cooling_degree_hours(resampled_rcmodel_dataf[schema.DataSchema.IAT], 24)
measured_cooling_degree_hours = calculate_cooling_degree_hours(measured_IAT[schema.DataSchema.IAT], 24)
error_cooling_degree_hours = (modelled_cooling_degree_hours-measured_cooling_degree_hours)/measured_cooling_degree_hours
print(f'number of modelled cooling degree hours {modelled_cooling_degree_hours}')
print(f'number of measured cooling degree hours {measured_cooling_degree_hours}')
print(f"The error in cooling degree hours of the model compared to measured data is {error_cooling_degree_hours:.2%}.")

In [ ]:
fhfgfh



In [ ]:
# Parameters 
path_results:Path = Path().absolute().parent/"data"
target_area:enums.Area = enums.Area.CARDIFF
target_year:int = 2020
CDD_ref_temperature:float = 24
solar_gains = True
timestep:int = 3600

# Dwelling parameters
R_dwelling = 1/0.2 #K/kW
C_dwelling = 20_000 #kJ/K
floor_area = 50 #m2
initial_indoor_air_temperature = 21

In [ ]:
# Load outdoor air temperature data for Cardiff
input_data = pd.read_csv(path_results/"raw"/f"{target_area.value}_degree_days.csv", index_col=0, parse_dates=True)
input_data

In [ ]:
# Load outdoor air temperature data for Cardiff
input_data = pd.read_csv(path_results/"raw"/f"{target_area.value}_degree_days.csv", index_col=0, parse_dates=True)

# Create models
dwelling = thermal_model.ThermalModel(R=R_dwelling, C=C_dwelling, initial_indoor_air_temperature=initial_indoor_air_temperature)
data_source = source.SimulationData(dwelling, input_data, timestep_simulation=3600)

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
dataf = data_source.filter_era5_data(list_years=[target_year])
ax2 = ax.twinx()
dataf[schema.OutputDataSchema.SOLARRADIATION].plot(ax=ax2, color=sns.color_palette()[1])
ax2.set_ylabel(schema.OutputDataSchema.SOLARRADIATION, color=sns.color_palette()[1])
dataf[schema.OutputDataSchema.OAT].plot(ax=ax, color=sns.color_palette()[0])
ax.set_ylabel(schema.OutputDataSchema.OAT, color=sns.color_palette()[0])

ax2.grid(False)
ax.set_title(f"{target_area.value}")
ax.legend().remove()
ax.margins(0, None)

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
dataf[schema.OutputDataSchema.HEATINGSEASON].plot(ax=ax)
ax.set_ylabel(schema.OutputDataSchema.HEATINGSEASON)

In [ ]:
rcmodel_dataf=data_source.create_era5_based_simulation_data(estimate_solar_gains=solar_gains, list_years=[target_year])
rcmodel_dataf = dwelling.estimate_heating_demand(rcmodel_dataf)
resampled_rcmodel_dataf = data_source.resample_modelling_results(rcmodel_dataf)
functions.print_heating_and_cooling_demand(resampled_rcmodel_dataf)

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
resampled_rcmodel_dataf[schema.DataSchema.SOLARGAINS].plot(ax=ax, color=sns.color_palette()[0])
ax.set_ylabel(schema.DataSchema.SOLARGAINS, color=sns.color_palette()[0])
ax.margins(0, None)

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
ax2 = ax.twinx()
resampled_rcmodel_dataf[schema.DataSchema.HEATINGOUTPUT].plot(ax=ax, color=sns.color_palette()[0])
ax.set_ylabel(schema.DataSchema.HEATINGOUTPUT, color=sns.color_palette()[0])
resampled_rcmodel_dataf[schema.DataSchema.IAT].plot(ax=ax2, color=sns.color_palette()[1])
ax2.set_ylabel(schema.DataSchema.IAT, color=sns.color_palette()[1])
ax2.set_ylim(0, 30)
ax2.grid(False)
ax.margins(0, None)

In [ ]:
resampled_rcmodel_dataf.index= dataf.index
resampled_rcmodel_dataf.to_csv(r"../data/demo_dwelling_results/simulation_results.csv")

In [ ]:

functions.add_time_features(resampled_rcmodel_dataf)

resampled_rcmodel_dataf[schema.OutputDataSchema.HEATINGSEASON] = dataf[schema.OutputDataSchema.HEATINGSEASON] 
resampled_rcmodel_dataf[schema.OutputDataSchema.SOLARRADIATION] = dataf[schema.OutputDataSchema.SOLARRADIATION] 

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
ax2 = ax.twinx()
filt = (resampled_rcmodel_dataf[schema.OutputDataSchema.HEATINGSEASON]==0)

(resampled_rcmodel_dataf.loc[filt].groupby("Hour").agg({schema.OutputDataSchema.SOLARRADIATION:"mean"})).plot(ax=ax2, color=sns.color_palette()[1])
(-resampled_rcmodel_dataf.loc[filt].groupby("Hour").agg({schema.DataSchema.HEATINGOUTPUT:"mean"})).plot(ax=ax, color=sns.color_palette()[0])
ax.set_ylabel("Cooling output (kW)", color=sns.color_palette()[0])
ax.legend().remove()
ax2.set_ylabel(schema.OutputDataSchema.SOLARRADIATION, color=sns.color_palette()[1])
ax2.grid(False)
ax.margins(0, None)

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
ax2 = ax.twinx()
filt = (resampled_rcmodel_dataf[schema.OutputDataSchema.HEATINGSEASON]==0)

resampled_rcmodel_dataf.loc[filt].groupby("Hour").agg({schema.DataSchema.IAT:"mean"}).plot(ax=ax2, color=sns.color_palette()[1])
resampled_rcmodel_dataf.loc[filt].groupby("Hour").agg({schema.DataSchema.OAT:"mean"}).plot(ax=ax2, color=sns.color_palette()[2])
(-resampled_rcmodel_dataf.loc[filt].groupby("Hour").agg({schema.DataSchema.HEATINGOUTPUT:"mean"})).plot(ax=ax, color=sns.color_palette()[0])
ax.set_ylabel("Cooling output (kW)", color=sns.color_palette()[0])
ax.legend().remove()
ax2.set_ylabel("Temperature")
ax2.grid(False)
ax.margins(0, None)



In [ ]:
resampled_rcmodel_dataf

In [ ]:
dataf

In [ ]:
results_dict = {}
heating_demand_dict = {}
for year in data_source.all_years[5:]:
    print(year)
    rcmodel_dataf=data_source.create_era5_based_simulation_data(estimate_solar_gains=solar_gains,list_years=[year])
    rcmodel_dataf = dwelling.estimate_heating_demand(rcmodel_dataf)
    resampled_rcmodel_dataf = data_source.resample_modelling_results(rcmodel_dataf)
    heating_demand, cooling_demand = functions.print_heating_and_cooling_demand(resampled_rcmodel_dataf)
    results_dict[year] = (heating_demand, cooling_demand)
    heating_demand_dict[year] = resampled_rcmodel_dataf.loc[:5088, schema.DataSchema.HEATINGOUTPUT].values
results_df = pd.DataFrame(results_dict, index=[schema.ResultSchema.HEATINGDEMAND, schema.ResultSchema.COOLINGDEMAND])
results_df.columns.names = [schema.ResultSchema.YEAR]

In [ ]:
profiles_df = pd.DataFrame.from_dict(heating_demand_dict)


In [ ]:
fig, ax = plt.subplots(figsize=figsize)
for c in profiles_df.columns:

    profiles_df.loc[3500:, c].rolling(window=48).mean().plot(ax=ax)

ax.legend()

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
ax2 = ax.twinx()
results_df.T[schema.ResultSchema.HEATINGDEMAND].plot(ax=ax, color=sns.color_palette()[0])
ax.set_ylabel(schema.ResultSchema.HEATINGDEMAND, color=sns.color_palette()[0])


x_arr = results_df.columns
y_mean = [results_df.T[schema.ResultSchema.COOLINGDEMAND].mean()]*len(x_arr)
ax2.plot(x_arr, y_mean, label=f"Mean {schema.ResultSchema.COOLINGDEMAND}", color='red')
results_df.T[schema.ResultSchema.COOLINGDEMAND].plot(ax=ax2, color=sns.color_palette()[1])
ax2.set_ylabel(schema.ResultSchema.COOLINGDEMAND, color=sns.color_palette()[1])
ax2.set_ylim(None, 0)
ax.set_ylim(0, None)
ax2.grid(False)
ax.margins(0, None)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)